In [1]:
import pandas as pd
import pandas_datareader.data as web
import datetime
from Indicator import Indicator

def fillindicator(df):
    indicator = Indicator()
    train_set = indicator.MA(df,15)
    train_set = indicator.MACD(df)
    train_set = indicator.STOCHASTIC(df)
    train_set = indicator.RSI(df)
    train_set = indicator.WILLIAMSR(df)
    return train_set

In [2]:
stockname = pd.read_csv('stock.csv')


In [3]:
def check_trend(df):
    df.loc[(df['Close'] > df['MA']) &
    ((df['Close'] > df['Close'].shift(1)) | (df['MA'] > df['MA'].shift(1))) &
    ((df['Close'] > df['Close'].shift(2)) | (df['MA'] > df['MA'].shift(2))) &
    ((df['Close'] > df['Close'].shift(3)) | (df['MA'] > df['MA'].shift(3))) &
    ((df['Close'] > df['Close'].shift(4)) | (df['MA'] > df['MA'].shift(4))) &
    ((df['Close'] > df['Close'].shift(5)) | (df['MA'] > df['MA'].shift(5))) , 'Trend'] = 'Up'

    df.loc[(df['Close'] < df['MA']) &
    ((df['Close'] < df['Close'].shift(1)) | (df['MA'] < df['MA'].shift(1))) &
    ((df['Close'] < df['Close'].shift(2)) | (df['MA'] < df['MA'].shift(2))) &
    ((df['Close'] < df['Close'].shift(3)) | (df['MA'] < df['MA'].shift(3))) &
    ((df['Close'] < df['Close'].shift(4)) | (df['MA'] < df['MA'].shift(4))) &
    ((df['Close'] < df['Close'].shift(5)) | (df['MA'] < df['MA'].shift(5))) , 'Trend'] = 'Down'
    
    df.loc[df['Trend'].isnull(), 'Ntrend'] = df['Trend'].shift(1)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull(),'Ntrend'] = df['Trend'].shift(2)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull(),'Ntrend'] = df['Trend'].shift(3)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull() & 
    df['Trend'].shift(3).isnull(),'Ntrend'] = df['Trend'].shift(4)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull() & 
    df['Trend'].shift(3).isnull() & df['Trend'].shift(4).isnull(),'Ntrend'] = df['Trend'].shift(5)
    df.loc[df['Trend'].isnull() & df['Trend'].shift(1).isnull() & df['Trend'].shift(2).isnull() & 
    df['Trend'].shift(3).isnull() & df['Trend'].shift(4).isnull() & df['Trend'].shift(5).isnull(),'Ntrend'] = df['Trend'].shift(6)
    
    df['S1'] = df['Close'].shift(-1)
    df['S2'] = df['Close'].shift(-2)
    df.loc[df['S1'].isnull(),'S1'] = 0
    df.loc[df['S2'].isnull(),'S2'] = 0

   
    #Trading Signal (0,1)
    df.loc[(df['Trend'] == 'Up') | (df['Ntrend'] == 'Up'), 'Trading Signal'] = ((abs(df['Close'] - df[['Close','S1','S2']].min(axis = 1)) / 
    abs(df[['Close','S1','S2']].max(axis = 1) - df[['Close','S1','S2']].min(axis = 1)))*0.5) + 0.5
    df.loc[(df['Trend'] == 'Down') | (df['Ntrend'] == 'Down'), 'Trading Signal'] = ((abs(df['Close'] - df[['Close','S1','S2']].min(axis = 1)) / 
    abs(df[['Close','S1','S2']].max(axis = 1) - df[['Close','S1','S2']].min(axis = 1)))*0.5)
    
    df = df.drop(['Trend','Ntrend','S1','S2'],axis=1)
        
    return df
    

In [4]:
start = datetime.datetime(2004,10,1)#26 day before start year for make indicator
end = datetime.datetime(2010,1,3)
for i,row in stockname.iterrows():
    try:
#         print(row['stock'])
        df = web.DataReader(row['stock'],'yahoo',start,end)
        df = fillindicator(df)
        df = check_trend(df)
        df =df.iloc[26:]
        df.to_csv('stock/'+row['stock']+'.csv')
    except:
        print(row['stock']+'  no data')

ALLE  no data
AMCR  no data
APTV  no data
ANET  no data
BRK.B  no data
BF.B  no data
CARR  no data
CTLT  no data
CBOE  no data
CDW  no data
CHTR  no data
CFG  no data
CTVA  no data
FANG  no data
DOW  no data
ETSY  no data
FB  no data
FRC  no data
FLT  no data
FTV  no data
FBHS  no data
FOXA  no data
FOX  no data
GM  no data
HCA  no data
HPE  no data
HLT  no data
HWM  no data
HII  no data
INFO  no data
IR  no data
IQV  no data
KEYS  no data
KMI  no data
KHC  no data
LW  no data
LUMN  no data
LYB  no data
MPC  no data
NWSA  no data
NWS  no data
NLSN  no data
NCLH  no data
OTIS  no data
PAYC  no data
PYPL  no data
PSX  no data
QRVO  no data
NOW  no data
SYF  no data
TWTR  no data
UA  no data
VIAC  no data
VNT  no data
WRK  no data
XYL  no data
ZTS  no data


In [5]:

df = web.DataReader(stockname['stock'][0],'yahoo',start,end)
df = fillindicator(df)
df = check_trend(df)
df =df.iloc[26:]

In [6]:
df.head(20)

,High,Low,Open,Close,Volume,Adj Close,MA,MACD,SIGNAL LINE,%K,%D,RSI-7,RSI-14,%R,Trading Signal
Date,,,,,,,,,,,,,,,
2004-11-05,81.620003,78.500000,78.500000,81.400002,6746600.0,52.979740,76.894000,0.041810,-0.289485,96.755147,77.698386,73.120455,63.625698,-3.244853,1.000000
2004-11-08,81.790001,80.830002,81.400002,81.379997,2969000.0,52.966690,77.246000,0.310583,-0.169239,94.100671,96.469848,72.943646,63.527729,-5.899329,1.000000
2004-11-09,81.849998,80.809998,81.629997,81.190002,2108300.0,52.843048,77.632667,0.502980,-0.034587,90.294169,93.716662,71.040221,62.542726,-9.705831,0.665051
2004-11-10,81.309998,80.540001,81.129997,80.849998,2871200.0,52.621769,77.902667,0.622645,0.097022,85.294108,89.896316,67.369906,60.728084,-14.705892,0.500000
2004-11-11,82.019997,80.910004,81.349998,81.879997,2547500.0,53.292133,78.188000,0.784224,0.234599,97.991399,91.193225,72.408151,64.123989,-2.008601,0.500000
2004-11-12,82.680000,81.900002,82.000000,82.680000,2676900.0,53.812828,78.641334,0.960290,0.379852,100.000000,94.428502,75.794809,66.543844,-0.000000,1.000000
2004-11-15,82.900002,82.029999,82.800003,82.279999,3054600.0,53.552475,79.088667,1.057191,0.515406,92.091806,96.694401,70.730567,64.211707,-7.908194,1.000000
2004-11-16,82.209999,81.570000,82.199997,81.769997,2825400.0,53.220528,79.458666,1.082779,0.628938,85.586680,92.559495,64.336336,61.263598,-14.413320,0.500000
2004-11-17,82.900002,81.699997,81.980003,82.230003,2819900.0,53.756611,79.769333,1.124740,0.728139,91.454109,89.710865,67.434327,62.917374,-8.545891,0.950705
